In [7]:
from elasticsearch import Elasticsearch
import pandas as pd

# Define the Elasticsearch client
es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme':'http'}])

In [13]:
index_name = "netflix_movie"
netflix_movie = es.indices.get(index=index_name)

In [9]:
netflix_movie

ObjectApiResponse({'netflix_movie': {'aliases': {}, 'mappings': {'_meta': {'created_by': 'file-data-visualizer'}, 'properties': {'cast': {'type': 'text'}, 'country': {'type': 'text'}, 'date_added': {'type': 'keyword'}, 'description': {'type': 'text'}, 'description_vector': {'type': 'dense_vector', 'dims': 768, 'index': True, 'similarity': 'l2_norm'}, 'director': {'type': 'text'}, 'duration': {'type': 'keyword'}, 'listed_in': {'type': 'text'}, 'rating': {'type': 'keyword'}, 'release_year': {'type': 'long'}, 'show_id': {'type': 'keyword'}, 'title': {'type': 'text'}, 'type': {'type': 'keyword'}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'netflix_movie', 'creation_date': '1716724277097', 'number_of_replicas': '1', 'uuid': '5pR_mfM7QAGkPEaThGAJoA', 'version': {'created': '8070199'}}}}})

In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('model')

In [18]:
# creating description embeddings for all the movies and updating the index with the embeddings
import tqdm

# retrieving all the movies
search_result = es.search(
  index=index_name,
  size=8805
)
for i in tqdm.tqdm(search_result['hits']['hits']):
    description = i['_source']['description']
    description_embedding = model.encode(description)
    es.update(index='netflix_movie', id=i['_id'], body={"doc": {"description_vector": description_embedding}})

100%|██████████| 8805/8805 [13:42<00:00, 10.70it/s]


In [21]:
input_keyword = "Blue shoes"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "description_vector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index=index_name, knn=query, source=["title", "description"])
res["hits"]["hits"]

C:\Users\ALFA RAYAN\AppData\Local\Temp\ipykernel_14272\898805795.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index=index_name, knn=query, source=["title", "description"])


[{'_index': 'netflix_movie',
  '_id': '18G9tI8BokgvgvJRoekg',
  '_score': 0.43314347,
  '_source': {'description': 'Family man Devin falls back into his sneaker obsession after his pal Bobby talks him into a wheeling-dealing scheme to score a mythical pair of kicks.',
   'title': 'Sneakerheads'}},
 {'_index': 'netflix_movie',
  '_id': 'AsG9tI8BokgvgvJRpf8t',
  '_score': 0.43014672,
  '_source': {'description': 'This documentary chronicles the rise of the footwear designer behind a billion-dollar empire and his subsequent comeback after a financial scandal.',
   'title': 'Maddman: The Steve Madden Story'}}]